In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import os
import sys
sys.path.append("..")
import minimise
# Enable or disable Tensor Float 32 Execution
tf.config.experimental.enable_tensor_float_32_execution(False)
import matplotlib.pyplot as plt
import scipy.constants as const


def write_profile(filename, centers, densities_O, densities_H):
    """
    Write the density profiles to a file.

    Parameters:
    - filename (str): Output file name.
    - centers (np.ndarray): Bin centers.
    - densities_O (np.ndarray): Density values for component O.
    - densities_H (np.ndarray): Density values for component H.
    """
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        writer.writerow(["xbins", "rho_O", "rho_H"])
        for center, density_O, density_H in zip(centers, densities_O, densities_H):
            writer.writerow([f"{center:.4f}", f"{density_O:.20f}", f"{density_H:.20f}"])


def LJ_wall_93(z_range, z_wall, epsilon, sigma, cutoff, place='lo'):
    """
    Calculate the Lennard-Jones wall potential for a range of z values.
 
    epsilon is in kT
    sigma is in Angstrom
    cutoff is in Angstrom
    """
 
    hilo = {'lo': 1.0, 'hi': -1.0}
    z_rel = hilo[place]*(z_range - z_wall)
    
    # Avoid division by zero and negative values in z_rel
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio3 = np.where(z_rel > 0, (sigma / z_rel) ** 3, 0)
        ratio9 = ratio3 ** 3
        energy = epsilon * (ratio9 * 2./15. - ratio3)
        
    # Apply boundary conditions
    energy[z_rel <= 0] = np.inf
    energy[z_rel > cutoff] = 0.0
    
    # Shift the potential to ensure it is zero at the cutoff
    cutoff_energy = epsilon * ((sigma / cutoff) ** 9 * 2./15. - (sigma / cutoff) ** 3)
    energy[z_rel <= cutoff] -= cutoff_energy
    
    return energy


def do_slit(mu, modelH_path, modelO_path, results_path):
    """
    Determine the self-consistent density profiles with neural DFT for all test systems.

    Parameters:
    - model_path (str): Path to the Keras model.
    - data_path (str): Path to the simulation data.
    - results_path (str): Path to save the results.
    """
    model_H = keras.models.load_model(modelH_path)
    model_O = keras.models.load_model(modelO_path)
    
    q_H = 1.0
    q_O = -1.0
    kappa_inv = 5.0
    temp = 4000.0
    dielectric = 1.0
  
    os.makedirs(results_path, exist_ok=True)

    
    output_file = f'{results_path}/slit_mu{mu}.out'
    
    zbins = np.arange(0, 30, 0.03)
 
    V_lo = LJ_wall_93(zbins, 5, 1.0, 1.0, 0.858374218933, place='lo')
    V_hi = LJ_wall_93(zbins, 25, 1.0, 1.0, 0.858374218933, place='hi')
    Vext = V_lo + V_hi
    muloc_H = - Vext + mu
    muloc_O = - Vext + mu    

    #muloc_H = -9.928 * np.ones_like(zbins) #- 0.15 * zbins + np.mean(0.15 * zbins)
    #muloc_O = -9.928 * np.ones_like(zbins) #+ 0.15 * zbins - np.mean(0.15 * zbins)
    
    muloc_H[zbins < 5] = np.inf
    muloc_O[zbins < 5] = np.inf
    muloc_H[zbins > 25] = np.inf
    muloc_O[zbins > 25] = np.inf
    
    zs, rho_H, rho_O = minimise.minimise_LR_twotype(model_H, model_O, zbins, muloc_H, muloc_O,
                                                    q_H, q_O, kappa_inv, temp, dielectric, 
                                                    initial_guess=0.014, input_bins=667, plot=True, 
                                                    tolerance=1.3e-5, maxiter=50000, tolerance_restr = 1e-5,
                                                    plot_every=100, print_every=100)
    if zs is not None:
        write_profile(output_file, zs, rho_H, rho_O)
        
    return zs, rho_H, rho_O


def do_slit_Efield(Efield, mu, modelH_path, modelO_path, results_path):
    """
    Determine the self-consistent density profiles with neural DFT for all test systems.

    Parameters:
    - Efield (float): Electric field, in real lammps unit, one would use add force q*Edfield
    - model_path (str): Path to the Keras model.
    - data_path (str): Path to the simulation data.
    - results_path (str): Path to save the results.
    """
    model_H = keras.models.load_model(modelH_path)
    model_O = keras.models.load_model(modelO_path)
    
    
    q_H = 1.0
    q_O = -1.0
    sigma = 5.0
    temp = 4000.0
    dielectric = 1.0
  
    os.makedirs(results_path, exist_ok=True)

    
    output_file = f'{results_path}/slit_mu{mu:.1f}_E{Efield:.1f}.out'
    
    zbins = np.arange(0, 200.03, 0.03)
     
    V_lo = LJ_wall_93(zbins, 80, 1.0, 1.0, 0.858374218933, place='lo')
    V_hi = LJ_wall_93(zbins, 120, 1.0, 1.0, 0.858374218933, place='hi')
    Vext = V_lo + V_hi
    
    newvalid = (80<zbins) & (zbins<120)
    
    conversion_from_lammps = const.Boltzmann * temp * const.Avogadro / 4184
    gradient = Efield / conversion_from_lammps
    ez = gradient * (zbins - 100)
    

    muloc_H = - Vext + q_H * ez + mu
    muloc_O = - Vext + q_O * ez + mu    

    
    
    zs, rho_H, rho_O = minimise.minimise_LR_twotype(model_H, model_O, zbins, muloc_H, muloc_O,
                                                    q_H, q_O, sigma, temp, dielectric, 
                                                    initial_guess=0.01, input_bins=667, plot=True, 
                                                    tolerance=1e-5, maxiter=50000, tolerance_restr = 1e-3,
                                                    plot_every=100, print_every=100)
    if zs is not None:
        write_profile(output_file, zs, rho_H, rho_O)
        
    return zs, rho_H, rho_O
    


In [ ]:

modelH_path = "../../models/RPM_H_Aug22.keras"
modelO_path = "../../models/RPM_O_Aug22.keras"
results_path = "../../tests/RPM_all_LR"

#z_range, rho_H, rho_O = do_slit(-10, modelH_path, modelO_path, results_path)


In [ ]:
#z_range, rho_H, rho_O = do_slit_Efield(5.0, -12, modelH_path, modelO_path, results_path)
